In [1]:
%matplotlib inline
%config InlineBackend.figure_format='retina'
import numpy as np
import shtools
import util
import lspn
import cas
import pnbuilder


In [2]:
def term1_subterm0_coeff_expr():
    omega = cas.tensor("\\omega", rank=1, dimension=3)
    omega_x = omega.getComponent(0)
    omega_y = omega.getComponent(1)
    omega_z = omega.getComponent(2)

    x = cas.tensor("\\vec{x}", rank=1, dimension=3)
    x.setComponent(0, cas.var("x"))
    x.setComponent(1, cas.var("y"))
    x.setComponent(2, cas.var("z"))
    x.collapsed = True

    L = cas.fun( "L", x, omega)

    sigma_t = cas.fun( "\\sigma_t", x)
    
    return cas.SHCoefficient( "L", cas.sub(cas.var("l'"), cas.num(1)), cas.sub(cas.var("m'"), cas.num(1)), x )
    #return cas.SHCoefficient( "L", cas.sub(cas.var("l'"), cas.num(0)), cas.sub(cas.var("m'"), cas.num(0)), x )

cas.print_expr(term1_subterm0_coeff_expr())


----------------------------

$$
L^{{l'-1,m'-1}}\left (\vec{x} \right )
$$


In [3]:

class Constant(object):
    def __init__(self, value):
        self.value = value
    def __call__(self, x):
        return self.value
    def dx(self, x):
        return 0.0
    def dy(self, x):
        return 0.0
    def dz(self, x):
        return 0.0

class Gradient(object):
    def __init__(self, normal):
        norm=np.linalg.norm(normal)
        if norm==0: 
            self.normal = normal
        else:
            self.normal = normal/norm
    def __call__(self, x):
        return np.abs(np.dot(x, self.normal))
    def dx(self, x):
        return self.normal[0]
    def dy(self, x):
        return self.normal[1]
    def dz(self, x):
        return 0.0
        #return self.normal[2]

    
class RBF(object):
    def __init__(self, stddev, amplitude=1.0 ):
        self.amplitude = amplitude
        self.stddev = stddev
        self.variance = stddev*stddev
        self.normalization = 1.0/(np.pow(stddev, 3.0)*np.pow(2.0*np.pi, 1.5))
    def __call__(self, x):
        return self.normalization*np.exp(-(x[0]*x[0]+x[1]*x[1]+x[2]*x[2])/self.variance)
    def dx(self, x):
        return 0.0
    def dy(self, x):
        return 0.0
    def dz(self, x):
        # NB: this is for our pseudo-2d case
        return 0.0

class SHEXP(object):
    def __init__(self, order, coeff_functions):
        self.order = order
        self.coeff_functions = coeff_functions
    def __call__(self, x, omega):
        (theta, phi) = shtools.sphericalCoordinates(omega)
        coeffs = [f(x) for f in self.coeff_functions]
        return shtools.sh_sum(self.order, coeffs, theta, phi)
        

In [4]:

res = 1
order = 1
domain = util.Domain2D(0.01, 1)
pnb = pnbuilder.PNBuilder(order, domain)

# here we compute the groundtruth for the given term. This means we actually compute the directional gradient of sigma_t
# and multiply this with L. The result is projected into spherical harmonics. This gives our groundtruth result
# after taking complex->real conversion into account

#sigma_t = Constant(1.0)
sigma_t = Gradient(np.array([1.0, 0.0]))
L = SHEXP(order, [Constant(1.0), Constant(0.0), Constant(0.0), Constant(0.0)])

def term1( x, theta, phi ):
    omega = shtools.sphericalDirection(theta, phi)
    return -(omega[0]*sigma_t.dx(x) + omega[1]*sigma_t.dy(x) + omega[2]*sigma_t.dz(x))*L(x, omega)
    
def term1_subterm0_coeff( x, l, m ):
    shindex = pnb.shIndex(l-1, m-1)
    if shindex is None:
        return 0.0
    return shtools.c_lm(l-1, m-1)*0.5*sigma_t.dx(x)*L.coeff_functions[shtools.shIndex(l-1, m-1)](x)
    
    

# here we project the term under investigation into SH. This is done at the respective locations
# of all the unknowns
coeffs_groundtruth_complex = []
for i in range(pnb.numCoeffs):
    voxel_i = 0
    voxel_j = 0
    pWS = pnb.get_unknown_location( voxel_i, voxel_j, i ).getPWS()
    #coeffs = shtools.project_sh(lambda theta, phi: term1(pWS, theta, phi), order)
    # NB: we take into account, that for 2d, pnb will have different index and lm ordering
    (l,m) = pnb.lmIndex(i)
    #coeffs_groundtruth_complex.append(coeffs[shtools.shIndex(l,m)])
    coeffs_groundtruth_complex.append(term1_subterm0_coeff(pWS, l, m))
    
# now convert from complex, to real-valued problem
coeffs_groundtruth_real = np.real(pnb.S.dot(coeffs_groundtruth_complex))


In [5]:
# now construct solution vector from known L at the
# respective coefficient locations
# this step is actually redundant, because L is already defined in terms of SH coefficients
# however, it serves as a sanity check...
x_complex = []
for i in range(pnb.numCoeffs):
    voxel_i = 0
    voxel_j = 0
    pWS = pnb.get_unknown_location( voxel_i, voxel_j, i ).getPWS()
    coeffs = shtools.project_sh(lambda theta, phi: L(pWS, shtools.sphericalDirection(theta, phi)), order)
    # NB: we take into account, that for 2d, pnb will have different index and lm ordering
    (l,m) = pnb.lmIndex(i)
    x_complex.append(coeffs[shtools.shIndex(l,m)])
            
# now convert from complex, to real-valued problem
x_real = np.real(pnb.S.dot(x_complex))

# we now apply A, which represents the effect of applying the term in question to L
#pnb.add_terms(lspn.lspn_extinction_directional_derivative_term(debug = False))
pnb.add_terms(term1_subterm0_coeff_expr())

problem = {}
problem["\\sigma_t"] = lambda pWS: sigma_t(pWS)
#problem["\\sigma_a"] = sigma_a
#problem["\\sigma_s"] = sigma_s
#problem["f_p"] = phase_shcoeffs
#problem["q"] = source_shcoeffs
(A,b) = pnb.build_global(problem)
y_real = A.dot(x_real)
y_complex = pnb.S_inv.dot(y_real)


building global systen Ax=b...
voxel_x=0


In [ ]:
#print(coeffs_groundtruth_complex)
#print(y_complex)

for i in range(pnb.numCoeffs):
    print("---")
    print("groundtruth: {} {}".format(np.real(coeffs_groundtruth_complex[i]), np.imag(coeffs_groundtruth_complex[i])))
    print("Ax         : {} {}".format(np.real(y_complex[i]), np.imag(y_complex[i])))

In [ ]:

for i in range(pnb.numCoeffs):
    print("{} {}".format(np.real(coeffs_groundtruth_complex[i]), np.imag(coeffs_groundtruth_complex[i])))
    print("{} {}".format(np.real(y_complex[i]), np.imag(y_complex[i])))

In [6]:
print(np.real(A))

[[ 0.          0.          0.        ]
 [-0.70710678  0.          0.        ]
 [ 0.          0.          0.        ]]


In [7]:
print(np.imag(A))

[[ 0.  0.  0.]
 [ 0.  0.  0.]
 [ 0.  0.  0.]]
